In [6]:
import cv2 as cv
import numpy as np
import pandas as pd
import PIL
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns

from evaluation.vlc import VLC
from utils.path_utils import Files
from evaluation.simulator import TimingConfig

# Analysis

## VLC

In [ ]:

directory = "/Users/guycohen/Downloads/eval/micro"
files = Files(directory, extension='png', sorting_key=lambda f: int(f.split('_')[-1].split('.')[0]))
timing_config = TimingConfig.load_json("/Users/guycohen/Downloads/eval/time_config.json")
log_path = "/Users/guycohen/Desktop/Bio-Project/Exp2/bboxes.csv"



In [ ]:
# vlc = VLC(files, timing_config, log_path, 'mic')
# vlc.mainloop()

## Plots

In [9]:
from evaluation.analysis import Plotter
original_log_path = "/Users/guycohen/Desktop/Bio-Project/eval/bboxes.csv"
log_path = "/Users/guycohen/Desktop/Bio-Project/TestController1/bboxes.csv"
timing_config = TimingConfig.load_json("/Users/guycohen/Desktop/Bio-Project/time_config.json")

pltr = Plotter(log_path, timing_config)

In [10]:
data = pltr.print_statistics()

##################### No Preds #####################
Num of No Preds: 3
No prediction in frames: [21997, 22004, 61199]
corresponding cycle steps: [7, 14, 14]
##################### Cycles #####################
Num of cycles: 4080
##################### Area Diff #####################
Non Perfect Predictions: 23.771%
##################### General #####################


,frame,cycle,plt_x,plt_y,cam_x,cam_y,cam_w,cam_h,mic_x,mic_y,...,wrm_y,wrm_w,wrm_h,wrm_center_x,wrm_center_y,mic_center_x,mic_center_y,wrm_speed,bbox_area_diff,bbox_edge_diff
count,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,61185.0,61185.0,61185.000000,61185.000000,...,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,61185.000000,6.118500e+04,61185.000000
mean,30607.000000,2040.000000,1266.066928,1121.144398,1266.066928,1121.144398,360.0,360.0,1436.066928,1291.144398,...,1294.445628,12.050242,13.215658,1446.019271,1301.053457,1446.066928,1301.144398,0.449628,2.864485e-02,0.485368
std,17662.732447,1177.515461,192.822860,215.186481,192.822860,215.186481,0.0,0.0,192.822860,215.186481,...,215.504895,3.073197,3.214409,193.070393,215.341397,192.822860,215.186481,17.426624,8.849963e-02,10.809388
min,15.000000,1.000000,881.000000,539.000000,881.000000,539.000000,360.0,360.0,1051.000000,709.000000,...,-1.000000,-1.000000,-1.000000,-1.500000,-1.500000,1061.000000,719.000000,0.000903,-2.349879e-14,0.000000
25%,15311.000000,1020.000000,1149.000000,1027.000000,1149.000000,1027.000000,360.0,360.0,1319.000000,1197.000000,...,1200.006134,9.541626,10.765381,1329.222527,1207.874611,1329.000000,1207.000000,0.135178,-3.912062e-15,0.000000
50%,30607.000000,2040.000000,1169.000000,1179.000000,1169.000000,1179.000000,360.0,360.0,1339.000000,1349.000000,...,1351.878448,11.856415,14.264297,1348.422417,1359.518929,1349.000000,1359.000000,0.233951,3.623678e-15,0.000000
75%,45903.000000,3060.000000,1490.000000,1261.000000,1490.000000,1261.000000,360.0,360.0,1660.000000,1431.000000,...,1434.784958,14.520340,15.803482,1669.565788,1440.429672,1670.000000,1441.000000,0.385014,1.615858e-14,0.000000
max,61199.000000,4079.000000,1555.000000,1390.000000,1555.000000,1390.000000,360.0,360.0,1725.000000,1560.000000,...,1560.643234,19.665741,19.883560,1729.797531,1569.973503,1735.000000,1570.000000,1999.702875,1.000000e+00,1722.000000


In [ ]:
data.columns

In [ ]:
def cond(df: pd.DataFrame, threshold: float=1.5) -> pd.DataFrame:
    mask1 = (df['cycle_step'] == 14) & (df['worm_center_dist'] > threshold)
    cycles = df[mask1]['cycle'].unique()
    return df['cycle'].isin(cycles)

pltr.plot_2d_deviation(hue='cycle_step')

In [ ]:
pltr.plot_deviation()

In [ ]:
pltr.plot_area_vs_speed_guy(n=3, window_size=10, hue='phase')

In [ ]:

fig, ax = plt.subplots()
sns.scatterplot(data=data, x='wrm_center_x', y='wrm_center_y', hue='frame', size='marker_size', ax=ax, alpha=0.3, linewidth=0.2)
ax.set_xlim(0)
ax.set_ylim(0)
ax.invert_yaxis()
fig.tight_layout()
plt.show()


In [ ]:
data['wrm_speed_avg'] = pltr.rolling_average(data, window_size=15, column='wrm_speed')

In [ ]:
data['marker_size'] = 0.1 + (data['bbox_area_diff'])*2

In [ ]:
data['marker_size'].describe()

In [ ]:
mask = pltr._data['wrm_w'] < 0
pltr._data.loc[mask, 'wrm_w'] = 0

mask = pltr._data['wrm_h'] < 0
pltr._data.loc[mask, 'wrm_h'] = 0

pltr._data['wrm_area'] = pltr._data['wrm_w'] * pltr._data['wrm_h']

In [ ]:
pltr._data['wrm_area'].describe()

In [ ]:
pltr._data[["wrm_w", "wrm_h", 'bbox_area_diff']].describe()

In [ ]:
(data['wrm_speed_avg'] > 2).sum()

In [ ]:
(data['bbox_area_diff'] > 0.2).sum()/len(data.index)

In [ ]:
pltr.remove_invalid_cycles(data)[["wrm_x", "wrm_y", "wrm_w", "wrm_h"]].describe()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=pltr._data, x='frame', y='wrm_speed_avg', hue='bbox_area_diff', ax=ax, s=8, linewidth=0.1)
ax.set_ylim(0, 2)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=pltr._data, x='wrm_speed_avg', y='bbox_area_diff', hue='wrm_area', ax=ax, s=8, linewidth=0.1)
ax.set_xlim(0, 2)

In [ ]:
data['residual'] = np.sqrt((data['wrm_center_x'] - data['mic_center_x'])**2 + (data['wrm_center_y'] - data['mic_center_y'])**2)

In [ ]:
data['residual'].describe()

In [ ]:
# data['diff_x'] = data['mic_x'].diff()
# data['diff_y'] = data['mic_y'].diff()
# sns.histplot(data=data, x='diff_y')

In [ ]:
data['cycle_step'] = data['frame']%15

In [ ]:
# fig, ax = plt.subplots()
# sns.jointplot(data=data, x='cycle_step', y='residual', ax=ax, s=8, linewidth=0.1)
sns.jointplot(data=data, x='cycle_step', y='wrm_speed', ax=ax, kind='hist')
# sns.histplot(data=data, x='cycle_step', y='residual')

In [ ]:
sns.catplot(data=data, x='cycle_step', y='wrm_speed', kind='violin')

In [ ]:
%matplotlib inline
for i in [1,2,4,8,15]:
    data = pltr.data_prep_frames(i)
    data['wrm_speed_avg'] = pltr.rolling_average(data, window_size=round(15/i), column='wrm_speed')
    # fig, ax = plt.subplots()
    mask = data['bbox_area_diff'] > 1e-3
    g = sns.jointplot(data=data[mask], x='wrm_speed_avg', y='bbox_area_diff', kind='scatter', xlim=(0, 3), dropna=True)
    g.fig.suptitle(f"i = {i}, rolling window = {round(15/i)}")


In [ ]:
plt._data.dtypes

# Run Simulation

In [1]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from dataset.raw_dataset import ExperimentConfig
from dummy_reader import DummyReader
from evaluation.motor_controller import *

base_path = "/Users/guycohen/Desktop/Bio-Project/"

## Run CSV Simulation

In [2]:
exp_base_path = base_path + "TestController1"

time_config = TimingConfig.load_json(base_path + "time_config.json")
exp_config = ExperimentConfig.load_json(base_path + "exp_config.json")
log_config = LogConfig(
    root_folder=exp_base_path,
    save_mic_view=False,
    save_cam_view=False,
)

In [3]:
from evaluation.sim_controllers.test_controller1 import TestController1
controller = TestController1(time_config, "/Users/guycohen/Desktop/Bio-Project/eval/bboxes.csv")

In [4]:
reader = DummyReader(exp_config)

logging_controller = LoggingController(controller, log_config)
motor_controller = SimpleMovementController(time_config)

sim = Simulator(time_config, exp_config, reader, logging_controller, motor_controller)

In [5]:
sim.run()

Simulation Progress:   0%|          | 0/4080 [00:00<?, ?cycle/s]

## Real Simulation

In [ ]:
"""experiment_config = ExperimentConfig(
    name="exp1",
    num_frames=61200,
    frames_per_sec=60,
    orig_resolution=(1920, 1080),
    px_per_mm=90,
    init_position=(1300, 1200),
)"""
# experiment_config.save_json("/Users/guycohen/Desktop/Bio-Project/exp_config.json")

In [ ]:

"""time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
    frame_padding_value=(255, 255, 255),
)

log_config = LogConfig(
    root_folder=base_path,
    save_mic_view=False,
    save_cam_view=False,
)"""

# yolo_config = YoloConfig(model_path="yolov8m-trained.pt") 

# time_config.save_json(base_path+"time_config.json")
# log_config.save_json(base_path+"csv_log_config.json")
# yolo_config.save_json(base_path++"yolo_config.json")


# frames_path = "Archive/"
# reader = FrameReader.create_from_directory(frames_path)

# time_config = TimingConfig.load_json("logs/time_config.json")
# log_config = LogConfig.load_json("logs/log_config.json")
# yolo_config = YoloConfig.load_json("logs/yolo_config.json")


In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [ ]:

controller = LoggingController(time_config, yolo_config, log_config)
#controller = YoloController(time_config, yolo_config)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run()

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes